# Исследование надежности заемщиков


<h3>Описание проекта</h3>

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.


<h3>Цель исследования</h3>

Влияние семейного положения и количества детей клиента на своевременное погашение кредита.

## Открытие данных




In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
# Загружаем данные
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

In [2]:
#изучим первые 10 строк
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
#Получим детальную информацию о колонках DataFrame
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [4]:
#Определяем, в каких столбцах есть пропуски.
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В столбце `days_employed` отрицательное кол-ство дней трудового стража - это аномалия. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится нам для исследования.

Выведем перечень уникальных значений столбца `children`.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждому типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены.

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [14]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.






In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`.

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

*Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.*


In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследование данных

**1. Посмотрим есть ли зависимость между количеством детей и возвратом кредита в срок**

In [22]:
def percent(data):
    return data.sum() / data.count() * 100
data_pivot = data.pivot_table(index=['children'], values='debt', aggfunc=['sum','count',percent])
data_pivot


,sum,count,percent
,debt,debt,debt
children,,,
0,1063,14091,7.543822
1,444,4808,9.234609
2,194,2052,9.454191
3,27,330,8.181818
4,4,41,9.756098
5,0,9,0.000000


**Вывод:**

По таблице видно, что у семей с 5 детьми вероятность неуплаты кредита в срок равна 0%, но и выборка слишком мала, потому скажем, что наиболее надежными заёмщиками являются семьи без детей, в то время, как семьи с 4 и 2 детьми являются наиболее ненадёжными.

**2. Посмотрим, есть ли зависимость между семейным положением и возвратом кредита в срок**

In [23]:
def percent(data):
    return data.sum() / data.count() * 100
data_pivot1 = data.pivot_table(index=['family_status'], values='debt', aggfunc=['sum','count',percent])
data_pivot1

,sum,count,percent
,debt,debt,debt
family_status,,,
Не женат / не замужем,273,2796,9.763948
в разводе,84,1189,7.064760
вдовец / вдова,63,951,6.624606
гражданский брак,385,4134,9.313014
женат / замужем,927,12261,7.560558


**Вывод:**

Самыми надежными заёмщиками являются вдовцы, самыми ненадежными одинокие люди. Так как здесь разница в процентах существенна, можно сказать, что да, семейное положение влияет на возврат кредита в срок.

**3. Посмотрим, есть ли зависимость между уровнем дохода и возвратом кредита в срок**

In [24]:
def percent(data):
    return data.sum() / data.count() * 100
data_pivot2 = data.pivot_table(index=['total_income_category'], values='debt', aggfunc=['sum','count',percent])
data_pivot2

,sum,count,percent
,debt,debt,debt
total_income_category,,,
A,2,25,8.000000
B,354,5014,7.060231
C,1353,15921,8.498210
D,21,349,6.017192
E,2,22,9.090909


**Вывод:**

 Самый большой процент неуплаты кредитов в срок у людей с доходом 50001–200000 - 8.49%
Самый низкий процент неуплаты кредитов в срок у людей с доходом 200001–1000000 - 7.06%
Брать в статистику другие группы уровней дохода мы не можем, так как слишком маленькая выборка, которая не будет являться достаточной для составления выводов.

**4. Посмотрим, как разные цели кредита влияют на его возврат в срок**

In [25]:
def percent(data):
    return data.sum() / data.count() * 100
data_pivot3 = data.pivot_table(index=['purpose_category'], values='debt', aggfunc=['sum','count',percent])
data_pivot3

,sum,count,percent
,debt,debt,debt
purpose_category,,,
операции с автомобилем,400,4279,9.347978
операции с недвижимостью,780,10751,7.255139
получение образования,369,3988,9.252758
проведение свадьбы,183,2313,7.911803


**Вывод:**

Чаще всего в срок возвращают кредиты взятые на недвижимость и проведение свадьбы, просрачивают чаще кредиты на автомобили и получение образования.

**Возможные причины появления пропусков в исходных данных.**

Скорее всего это был человеческий фактор, так как данные пропущены лишь в 2 столбцах с указанием общего трудового стажа и ежемесячного дохода. Возможно, у людей не было официального трудоустройства, они не предоставили справки с работы и потому эти данные не были внесены в таблицы.

### Шаг 4: общий вывод.


Наиболее надежными заёмщиками для банка являются люди:
- без детей, так как с ростом количества детей, вероятность неуплаты кредиты в срок так же возрастает.
- вдовцы или же женатые, то есть клиенты, побывавшие в браке более ответственны к оплате кредитов в срок, чем свободные клиенты.
- с доходом 200001–1000000, соответсвенно чем ниже доход, тем выше вероятность того, что клиент не будет возвращать долг во время.
- берущие кредит на недвижимость или проведение свадьбы, кредиты на автомобили и получение образования чаще не возвращают в срок.